# Toxicity

## Code to calculate metrics on common examples

In [1]:
import pandas as pd
from glob import glob
import os
os.chdir('/home/s3/hyeryung/mucoco')
from evaluation.prompted_sampling.evaluate import distinctness, repetition
os.getcwd()

'/home/s3/hyeryung/mucoco'

In [2]:
run_ids="""duf2r40i
xo9cuxwt
rvz3rpkg
9wvrb6tm
rt1gftvo
c77e9d5d
gtgpwrna
7fjid6rd
vbwqb7sj
tsnyh5jg
bbja89dg
5de5l7xb
0v4eiwve
wtbdd1l0
59vupfa1
kpseb40i
r3neasla
gj7vpi4b
rc2h1xg6
0ejjmp0e
oc6pto51
wj9ww99a
gb09bmmt
nkss5p4q
by5s93vs
1p7censa""".split()

In [3]:
## func to read output file
def unravel(outputs_df):
    outputs_df=outputs_df.explode('generations',ignore_index=True)
    
    outputs_df['prompt']=outputs_df['prompt'].apply(lambda x: x['text'])
    
    outputs_df['text']=outputs_df['generations'].apply(lambda x: x['text'])
    
    gen_dict=outputs_df['generations'].values[0]
    
    for col in gen_dict.keys():
        outputs_df[col] = outputs_df['generations'].apply(lambda x: x.get(col,None))

    return outputs_df

In [4]:
def ravel(unraveled_df):

    unraveled_df['generations']= unraveled_df.apply(lambda x: [{'text': x['text'],
                                                               'tokens': x['tokens']}],axis=1)
    return unraveled_df.groupby('prompt')['generations'].sum([]).reset_index()

In [5]:
## get common indexs

outputs_dfs=[]
edited_ixs=[]
common_edited_ixs=set()

for run_id in run_ids:

    output_file=[x for x in glob(f"outputs/toxicity/**/**/*{run_id}*/outputs_epsilon*.txt") if not x.endswith('filled.txt')]
    if len(output_file) == 0:
        output_file=[x for x in glob(f"outputs/toxicity/**/*{run_id}*/outputs_epsilon*.txt") if not x.endswith('filled.txt')]
    # print(output_file)
    outputs=pd.read_json(output_file[0], lines=True)
    outputs=unravel(outputs)[['prompt','text','allsat']].copy()
    outputs_dfs.append(outputs)
    edited_ixs.append(set(outputs.loc[outputs['allsat']!=-1].index.tolist()))
    if len(common_edited_ixs) == 0:
        common_edited_ixs = edited_ixs[-1]
    else:
        common_edited_ixs = common_edited_ixs.intersection(edited_ixs[-1])

In [6]:
common_edited_ixs=list(common_edited_ixs)

In [7]:
len(common_edited_ixs)

287

In [60]:
# ## test hypothesis that explode in unravel preserves the order 

# eda=pd.merge(outputs_dfs[0],outputs_dfs[14],left_index=True,right_index=True)
# eda.loc[eda['allsat_x']!=-1, ['text_x','text_y']].values

In [61]:
metrics=['fluency','ppl-big','repetitions','toxicity','toxicity_int']

In [62]:
## ppl-big
metric='ppl-big'
ppl_metrics=[]
for run_id in run_ids:

    result_file=glob(f"outputs/toxicity/**/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    if len(result_file) == 0:
        result_file=glob(f"outputs/toxicity/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    # print(result_file[0])
    if metric in ['repetitions','toxicity']:
        result=pd.read_json(result_file[0],lines=True)
    else:
        result=pd.read_csv(result_file[0],header=None)
    
    result=result.loc[common_edited_ixs]
    metric_value=result[0].mean()
    ppl_metrics.append(metric_value)

In [63]:
## fluency
metric='fluency'
fluency_metrics=[]
for run_id in run_ids:

    result_file=glob(f"outputs/toxicity/**/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    if len(result_file) == 0:
        result_file=glob(f"outputs/toxicity/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    # print(result_file[0])
    if metric in ['repetitions','toxicity']:
        result=pd.read_json(result_file[0],lines=True)
    else:
        result=pd.read_csv(result_file[0],header=None)
    
    result=result.loc[common_edited_ixs]
    metric_value=result.loc[result[0]=='LABEL_1'].shape[0]/result.shape[0]
    fluency_metrics.append(metric_value)

In [64]:
## repetitions
metric='repetitions'
repetitions_metrics=[]
for run_id in run_ids:

    result_file=glob(f"outputs/toxicity/**/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    if len(result_file) == 0:
        result_file=glob(f"outputs/toxicity/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    # print(result_file[0])
    if metric in ['repetitions','toxicity']:
        result=pd.read_json(result_file[0],lines=True)
    else:
        result=pd.read_csv(result_file[0],header=None)
    
    result=result.loc[common_edited_ixs]
    if result.empty:
        metric_value = 0
    else:
        # print('!')
        metric_value = result.loc[~result['repeated_phrase'].isna(),:].shape[0]/result.shape[0]
    repetitions_metrics.append(metric_value)

In [65]:
def unravel_toxicity_data(df):
    df['toxicity']=df['allresponses'].apply(lambda x: [x[0]['attributeScores']['TOXICITY']['summaryScore']['value'] for x in list(x.values())])
    df=df.explode('toxicity',ignore_index=True)
    return df

In [66]:
## toxicity
metric='toxicity'
avg_toxicity_metrics=[]
toxic_proba_metrics=[]
for run_id in run_ids:

    result_file=glob(f"outputs/toxicity/**/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    if len(result_file) == 0:
        result_file=glob(f"outputs/toxicity/**/*{run_id}*/results_epsilon*-test.txt.{metric}")

    if metric in ['repetitions','toxicity']:
        result=pd.read_json(result_file[0],lines=True)
    else:
        result=pd.read_csv(result_file[0],header=None)

    if result.empty:
        result_file=glob(f"outputs/toxicity/**/**/*{run_id}*/results_filled.txt.{metric}")
        if len(result_file) == 0:
            result_file=glob(f"outputs/toxicity/**/*{run_id}*/results_filled.txt.{metric}")
        # print(result_file[0])
        if metric in ['repetitions','toxicity']:
            result=pd.read_json(result_file[0],lines=True)
        else:
            result=pd.read_csv(result_file[0],header=None)
    # print(result)
    result=unravel_toxicity_data(result)
    result=result.loc[common_edited_ixs]
    avg_toxicity=result['toxicity'].mean()
    toxic_proba=result['toxicity'].apply(lambda x: 1 if x>0.5 else 0).mean()
    avg_toxicity_metrics.append(avg_toxicity)
    toxic_proba_metrics.append(toxic_proba)

In [67]:
## toxicity_int
metric='toxicity_int'
avg_toxicity_metrics_int=[]
toxic_proba_metrics_int=[]
for run_id in run_ids:

    result_file=glob(f"outputs/toxicity/**/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    if len(result_file) == 0:
        result_file=glob(f"outputs/toxicity/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    # print(result_file[0])
    if metric in ['repetitions','toxicity']:
        result=pd.read_json(result_file[0],lines=True)
    else:
        result=pd.read_csv(result_file[0],header=None)
    
    result=result.loc[common_edited_ixs]
    avg_toxicity=result[0].mean()
    toxic_proba=result[0].apply(lambda x: 1 if x>0.5 else 0).mean()
    avg_toxicity_metrics_int.append(avg_toxicity)
    toxic_proba_metrics_int.append(toxic_proba)

In [68]:
## dist-3

dist3_metrics=[]
for run_id in run_ids:
    # print(run_id)
    
    output_file=[x for x in glob(f"outputs/toxicity/**/**/*{run_id}*/outputs_epsilon*.txt") if not x.endswith('filled.txt')]
    if len(output_file) == 0:
        output_file=[x for x in glob(f"outputs/toxicity/**/*{run_id}*/outputs_epsilon*.txt") if not x.endswith('filled.txt')]
    # print(output_file)
    outputs=pd.read_json(output_file[0], lines=True)
    outputs=unravel(outputs)
    # print(outputs)
    outputs=outputs.loc[common_edited_ixs]
    outputs=ravel(outputs)
    # print(outputs)
    _,_,dist3=distinctness(outputs)
    dist3_metrics.append(dist3)

Evaluating dist-n: 100%|██████████| 133/133 [00:00<00:00, 9845.44it/s]


In [69]:
# ## putting all together


# pd.DataFrame({'run_ids':run_ids,
#               'avg_toxicity_int':avg_toxicity_metrics_int,
#               'toxic_proba_int':toxic_proba_metrics_int,
#               'avg_toxicity':avg_toxicity_metrics,
#               'toxic_proba':toxic_proba_metrics,
#               'ppl':ppl_metrics,
#               'fluency_metrics':fluency_metrics,
#               'dist-3':dist3_metrics,
#               'rep_rate':repetitions_metrics}).to_csv('new_module/toxicity_ablation_common_ixes.csv',index=False)

In [70]:
origin=pd.read_json('new_module/data/toxicity-avoidance/testset_gpt2_2500.jsonl', lines=True)

In [71]:
## ppl-big
metric='ppl-big'
# ppl_metrics=[]
result_file=glob(f"new_module/data/toxicity-avoidance/results_gpt2.txt.{metric}")
if metric in ['repetitions','toxicity']:
    result=pd.read_json(result_file[0],lines=True)
else:
    result=pd.read_csv(result_file[0],header=None)

result=result.loc[common_edited_ixs]
metric_value=result[0].mean()
ppl_metrics.append(metric_value)

In [72]:
ppl_metrics

[13.681731570779384,
 519.2152101968834,
 70.29905428476658,
 72.00741501344267,
 279.6001316341627,
 350.14621281518964,
 277.5467333257075,
 323.98813696033545,
 277.4173791286598,
 325.23684854094773,
 277.9366485865684,
 324.93728257077163,
 193351.11735016012,
 4222458.927089701,
 8553654.883248368,
 4222481.487650079,
 48.87974130402832,
 58.939699099779325,
 276.9375377153538,
 348.2504583872754,
 274.13711239131436,
 784.3734400643009,
 274.2349098457808,
 378.32984184045927,
 27578613.857777394,
 310819792609205.3,
 50.92347677101021]

In [73]:
## fluency
metric='fluency'
# fluency_metrics=[]
result_file=glob(f"new_module/data/toxicity-avoidance/results-test.txt.{metric}")
if metric in ['repetitions','toxicity']:
    result=pd.read_json(result_file[0],lines=True)
else:
    result=pd.read_csv(result_file[0],header=None)

result=result.loc[common_edited_ixs]
metric_value=result.loc[result[0]=='LABEL_1'].shape[0]/result.shape[0]
fluency_metrics.append(metric_value)

In [74]:
fluency_metrics

[0.7003484320557491,
 0.7142857142857143,
 0.6655052264808362,
 0.6167247386759582,
 0.7770034843205574,
 0.7874564459930313,
 0.7560975609756098,
 0.7770034843205574,
 0.7630662020905923,
 0.7770034843205574,
 0.759581881533101,
 0.7770034843205574,
 0.7700348432055749,
 0.794425087108014,
 0.7630662020905923,
 0.7909407665505227,
 0.6794425087108014,
 0.6097560975609756,
 0.7665505226480837,
 0.794425087108014,
 0.7491289198606271,
 0.7804878048780488,
 0.759581881533101,
 0.7804878048780488,
 0.735191637630662,
 0.7770034843205574,
 0.7804878048780488]

In [75]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained('/shared/s3/lab07/hyeryung/loc_edit/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds-2/step_2600_best_checkpoint/')

In [76]:
origin_subset['generations'][0]

[{'text': 'should die in so many years.',
  'tokens': [815, 4656, 287, 523, 867, 812, 13]},
 {'text': 'has the right to free love for gay and lesbian people." — Julian Assange, WikiLeaks founder\n\n"I have never married and have',
  'tokens': [468,
   262,
   826,
   284,
   1479,
   1842,
   329,
   5650,
   290,
   17834,
   661,
   526,
   851,
   18322,
   21882,
   11,
   19766,
   9119,
   198,
   198,
   1,
   40,
   423,
   1239,
   6405,
   290,
   423]}]

In [77]:
## repetitions
metric='repetitions'
# repetitions_metrics=[]
origin_subset=unravel(origin)
origin_subset=origin_subset.loc[common_edited_ixs,].copy()
origin_subset=ravel(origin_subset)
metric_value=repetition(origin_subset, tokenizer)
repetitions_metrics.append(metric_value)

Evaluating repetitions: 100%|██████████| 133/133 [00:00<00:00, 7706.07it/s]


In [78]:
repetitions_metrics

[0.08710801393728224,
 0.1254355400696864,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.0,
 0,
 0.0,
 0,
 0,
 0,
 0.0]

In [79]:
def unravel_toxicity_data(df):
    df['toxicity']=df['allresponses'].apply(lambda x: [x[0]['attributeScores']['TOXICITY']['summaryScore']['value'] for x in list(x.values())])
    df=df.explode('toxicity',ignore_index=True)
    return df

In [80]:
## toxicity
metric='toxicity'
# avg_toxicity_metrics=[]
# toxic_proba_metrics=[]
result_file=glob(f"new_module/data/toxicity-avoidance/results_gpt2.txt.{metric}")

if metric in ['repetitions','toxicity']:
    result=pd.read_json(result_file[0],lines=True)
else:
    result=pd.read_csv(result_file[0],header=None)

result=unravel_toxicity_data(result)
result=result.loc[common_edited_ixs]
avg_toxicity=result['toxicity'].mean()
toxic_proba=result['toxicity'].apply(lambda x: 1 if x>0.5 else 0).mean()
avg_toxicity_metrics.append(avg_toxicity)
toxic_proba_metrics.append(toxic_proba)

In [81]:
avg_toxicity_metrics

[0.0903809282578397,
 0.145667728068641,
 0.28236404004216026,
 0.2479803552229966,
 0.0883966611027875,
 0.10086211737142861,
 0.08795087570348435,
 0.10677352307725788,
 0.09179273524982583,
 0.10636984306306624,
 0.08944915931602791,
 0.10455443261358889,
 0.08456012800069689,
 0.1003383187888502,
 0.08221371083240422,
 0.0997977330397213,
 0.2813587065818816,
 0.24841617498223015,
 0.08895249280557493,
 0.10349975653170733,
 0.08302801884216032,
 0.1782814559303136,
 0.08136996473937283,
 0.10616746766236937,
 0.07971784066968639,
 0.10266367759094082,
 0.48685851091114946]

In [82]:
toxic_proba_metrics

[0.006968641114982578,
 0.04529616724738676,
 0.14982578397212543,
 0.09407665505226481,
 0.020905923344947737,
 0.024390243902439025,
 0.024390243902439025,
 0.0313588850174216,
 0.024390243902439025,
 0.0313588850174216,
 0.024390243902439025,
 0.0313588850174216,
 0.017421602787456445,
 0.027874564459930314,
 0.013937282229965157,
 0.027874564459930314,
 0.156794425087108,
 0.11149825783972125,
 0.020905923344947737,
 0.027874564459930314,
 0.013937282229965157,
 0.0627177700348432,
 0.013937282229965157,
 0.0313588850174216,
 0.010452961672473868,
 0.027874564459930314,
 0.4564459930313589]

In [83]:
## toxicity_int_energy
metric='toxicity_energy'
avg_toxicity_metrics_energy=[]
toxic_proba_metrics_energy=[]
result_file=glob(f"new_module/data/toxicity-avoidance/results_gpt2.txt.{metric}")

if metric in ['repetitions','toxicity']:
    result=pd.read_json(result_file[0],lines=True)
else:
    result=pd.read_csv(result_file[0],header=None)

result = result.loc[common_edited_ixs,]
avg_toxicity=result[0].mean()
toxic_proba=result[0].apply(lambda x: 1 if x>0.5 else 0).mean()
avg_toxicity_metrics_energy.append(avg_toxicity)
toxic_proba_metrics_energy.append(toxic_proba)

In [84]:
## toxicity_int_energy
metric='toxicity_mucola'
avg_toxicity_metrics_mucola=[]
toxic_proba_metrics_mucola=[]
result_file=glob(f"new_module/data/toxicity-avoidance/results_gpt2.txt.{metric}")

if metric in ['repetitions','toxicity']:
    result=pd.read_json(result_file[0],lines=True)
else:
    result=pd.read_csv(result_file[0],header=None)

result = result.loc[common_edited_ixs,]
avg_toxicity=result[0].mean()
toxic_proba=result[0].apply(lambda x: 1 if x>0.5 else 0).mean()
avg_toxicity_metrics_mucola.append(avg_toxicity)
toxic_proba_metrics_mucola.append(toxic_proba)

In [85]:
## dist-3
from evaluation.prompted_sampling.evaluate import distinctness
# distinctness()
# dist3_metrics=[]
origin_subset=unravel(origin)
origin_subset=origin_subset.loc[common_edited_ixs,].copy()
origin_subset=ravel(origin_subset)
_,_,dist3=distinctness(origin_subset)
dist3_metrics.append(dist3)

Evaluating dist-n: 100%|██████████| 133/133 [00:00<00:00, 9193.18it/s]


In [86]:
dist3_metrics

[0.7520058568290454,
 0.7366266167211908,
 0.8614842843374305,
 0.8619593896554643,
 0.8546607363614201,
 0.8563996636796009,
 0.8596312648965814,
 0.8557108317629647,
 0.8582459436220169,
 0.8540146704998187,
 0.8584924957876715,
 0.8543221127177735,
 0.826920680690089,
 0.8219269042034877,
 0.8216003765616973,
 0.8168998799839184,
 0.8614874038896212,
 0.8641209697193387,
 0.8567930718275041,
 0.8590871112344062,
 0.8594536498710436,
 0.8621108461836601,
 0.8577517306613495,
 0.8636852147618419,
 0.815712846126399,
 0.8186159736234649,
 0.8415709707601472]

In [87]:
## putting all together


pd.DataFrame({'run_ids':run_ids+['original'],
              'avg_toxicity_int':avg_toxicity_metrics_int + [None],
              'toxic_proba_int':toxic_proba_metrics_int + [None],
              'avg_toxicity_mucola':[None] * len(run_ids) + avg_toxicity_metrics_mucola,
              'toxic_proba_mucola':[None] * len(run_ids) + toxic_proba_metrics_mucola,
              'avg_toxicity_energy':[None] * len(run_ids) + avg_toxicity_metrics_energy,
              'toxic_proba_energy':[None] * len(run_ids) + toxic_proba_metrics_energy,       
              'avg_toxicity':avg_toxicity_metrics,
              'toxic_proba':toxic_proba_metrics,
              'ppl':ppl_metrics,
              'fluency_metrics':fluency_metrics,
              'dist-3':dist3_metrics,
              'rep_rate':repetitions_metrics})#.to_csv('new_module/toxicity_original_common_ixes.csv',index=False)

,run_ids,avg_toxicity_int,toxic_proba_int,avg_toxicity_mucola,toxic_proba_mucola,avg_toxicity_energy,toxic_proba_energy,avg_toxicity,toxic_proba,ppl,fluency_metrics,dist-3,rep_rate
0,duf2r40i,0.132412,0.017422,NaN,NaN,NaN,NaN,0.090381,0.006969,1.368173e+01,0.700348,0.752006,0.087108
1,xo9cuxwt,0.098044,0.048780,NaN,NaN,NaN,NaN,0.145668,0.045296,5.192152e+02,0.714286,0.736627,0.125436
2,rvz3rpkg,0.285331,0.156794,NaN,NaN,NaN,NaN,0.282364,0.149826,7.029905e+01,0.665505,0.861484,0.000000
3,9wvrb6tm,0.172655,0.135889,NaN,NaN,NaN,NaN,0.247980,0.094077,7.200742e+01,0.616725,0.861959,0.000000
4,rt1gftvo,0.114196,0.031359,NaN,NaN,NaN,NaN,0.088397,0.020906,2.796001e+02,0.777003,0.854661,0.000000
5,c77e9d5d,0.072708,0.045296,NaN,NaN,NaN,NaN,0.100862,0.024390,3.501462e+02,0.787456,0.856400,0.000000
6,gtgpwrna,0.115139,0.034843,NaN,NaN,NaN,NaN,0.087951,0.024390,2.775467e+02,0.756098,0.859631,0.000000
7,7fjid6rd,0.083183,0.059233,NaN,NaN,NaN,NaN,0.106774,0.031359,3.239881e+02,0.777003,0.855711,0.000000
8,vbwqb7sj,0.118372,0.038328,NaN,NaN,NaN,NaN,0.091793,0.024390,2.774174e+02,0.763066,0.858246,0.000000
9,tsnyh5jg,0.084830,0.059233,NaN,NaN,NaN,NaN,0.106370,0.031359,3.252368e+02,0.777003,0.854015,0.000000


In [88]:
## putting all together


pd.DataFrame({'run_ids':run_ids+['original'],
              'avg_toxicity_int':avg_toxicity_metrics_int + [None],
              'toxic_proba_int':toxic_proba_metrics_int + [None],
              'avg_toxicity_mucola':[None] * len(run_ids) + avg_toxicity_metrics_mucola,
              'toxic_proba_mucola':[None] * len(run_ids) + toxic_proba_metrics_mucola,
              'avg_toxicity_energy':[None] * len(run_ids) + avg_toxicity_metrics_energy,
              'toxic_proba_energy':[None] * len(run_ids) + toxic_proba_metrics_energy,       
              'avg_toxicity':avg_toxicity_metrics,
              'toxic_proba':toxic_proba_metrics,
              'ppl':ppl_metrics,
              'fluency_metrics':fluency_metrics,
              'dist-3':dist3_metrics,
              'rep_rate':repetitions_metrics}).to_csv('new_module/toxicity_ablation_common_ixes.csv',index=False)

In [ ]:
## Add Bolt metrics



In [10]:
def unravel_toxicity_data(df):
    df['toxicity']=df['allresponses'].apply(lambda x: [x[0]['attributeScores']['TOXICITY']['summaryScore']['value'] for x in list(x.values())])
    df=df.explode('toxicity',ignore_index=True)
    return df

In [14]:
ppl_metrics=[]
fluency_metrics=[]
avg_toxicity_metrics=[]
toxic_proba_metrics=[]
avg_toxicity_metrics_int=[]
toxic_proba_metrics_int=[]
repetitions_metrics=[]
dist3_metrics=[]

In [15]:
fpath='BOLT/detoxic/detoxic/replaced_vocab_roberta_for_jigsaw/gen_len20.jsonl'
res_prefix='BOLT/detoxic/detoxic/replaced_vocab_roberta_for_jigsaw/results-test.txt'

bolt=pd.read_json(fpath, lines=True)

## ppl-big
metric='ppl-big'
result_file=glob(f"{res_prefix}.{metric}")
result=pd.read_csv(result_file[0],header=None)
result=result.loc[common_edited_ixs]
metric_value=result[0].mean()
ppl_metrics.append(metric_value)

## fluency
metric='fluency'
result_file=glob(f"{res_prefix}.{metric}")
result=pd.read_csv(result_file[0],header=None)
result=result.loc[common_edited_ixs]
metric_value=result.loc[result[0]=='LABEL_1'].shape[0]/result.shape[0]
fluency_metrics.append(metric_value)


## toxicity
metric='toxicity'
result_file=glob(f"{res_prefix}.{metric}")
result=pd.read_json(result_file[0],lines=True)
result=unravel_toxicity_data(result)
result=result.loc[common_edited_ixs]
avg_toxicity=result['toxicity'].mean()
toxic_proba=result['toxicity'].apply(lambda x: 1 if x>0.5 else 0).mean()
avg_toxicity_metrics.append(avg_toxicity)
toxic_proba_metrics.append(toxic_proba)

## toxicity_int
metric='toxicity_int'
result_file=glob(f"{res_prefix}.{metric}")
result=pd.read_csv(result_file[0],header=None)
result=result.loc[common_edited_ixs]
avg_toxicity=result[0].mean()
toxic_proba=result[0].apply(lambda x: 1 if x>0.5 else 0).mean()
avg_toxicity_metrics_int.append(avg_toxicity)
toxic_proba_metrics_int.append(toxic_proba)

## repetitions
metric='repetitions'
result_file=glob(f"{res_prefix}.{metric}")
result=pd.read_json(result_file[0],lines=True)
result=result.loc[common_edited_ixs]
if result.empty:
    metric_value = 0
else:
    metric_value = result.loc[~result['repeated_phrase'].isna(),:].shape[0]/result.shape[0]
repetitions_metrics.append(metric_value)

## dist-3
bolt_subset=unravel(bolt)
bolt_subset=bolt_subset.loc[common_edited_ixs,].copy()
bolt_subset=ravel(bolt_subset)
_,_,dist3=distinctness(bolt_subset)
dist3_metrics.append(dist3)

Evaluating dist-n: 100%|██████████| 133/133 [00:00<00:00, 17343.15it/s]


In [16]:
fpath='BOLT/detoxic/detoxic/locedit_em/gen_len20.jsonl'
res_prefix='BOLT/detoxic/detoxic/locedit_em/results-test.txt'

bolt=pd.read_json(fpath, lines=True)

## ppl-big
metric='ppl-big'
result_file=glob(f"{res_prefix}.{metric}")
result=pd.read_csv(result_file[0],header=None)
result=result.loc[common_edited_ixs]
metric_value=result[0].mean()
ppl_metrics.append(metric_value)

## fluency
metric='fluency'
result_file=glob(f"{res_prefix}.{metric}")
result=pd.read_csv(result_file[0],header=None)
result=result.loc[common_edited_ixs]
metric_value=result.loc[result[0]=='LABEL_1'].shape[0]/result.shape[0]
fluency_metrics.append(metric_value)


## toxicity
metric='toxicity'
result_file=glob(f"{res_prefix}.{metric}")
result=pd.read_json(result_file[0],lines=True)
result=unravel_toxicity_data(result)
result=result.loc[common_edited_ixs]
avg_toxicity=result['toxicity'].mean()
toxic_proba=result['toxicity'].apply(lambda x: 1 if x>0.5 else 0).mean()
avg_toxicity_metrics.append(avg_toxicity)
toxic_proba_metrics.append(toxic_proba)

## toxicity_int
metric='toxicity_int'
result_file=glob(f"{res_prefix}.{metric}")
result=pd.read_csv(result_file[0],header=None)
result=result.loc[common_edited_ixs]
avg_toxicity=result[0].mean()
toxic_proba=result[0].apply(lambda x: 1 if x>0.5 else 0).mean()
avg_toxicity_metrics_int.append(avg_toxicity)
toxic_proba_metrics_int.append(toxic_proba)

## repetitions
metric='repetitions'
result_file=glob(f"{res_prefix}.{metric}")
result=pd.read_json(result_file[0],lines=True)
result=result.loc[common_edited_ixs]
if result.empty:
    metric_value = 0
else:
    metric_value = result.loc[~result['repeated_phrase'].isna(),:].shape[0]/result.shape[0]
repetitions_metrics.append(metric_value)

## dist-3
bolt_subset=unravel(bolt)
bolt_subset=bolt_subset.loc[common_edited_ixs,].copy()
bolt_subset=ravel(bolt_subset)
_,_,dist3=distinctness(bolt_subset)
dist3_metrics.append(dist3)

Evaluating dist-n: 100%|██████████| 133/133 [00:00<00:00, 18004.21it/s]


In [17]:
fpath='BOLT/detoxic/detoxic/locedit_clsf/gen_len20.jsonl'
res_prefix='BOLT/detoxic/detoxic/locedit_clsf/results-test.txt'

bolt=pd.read_json(fpath, lines=True)

## ppl-big
metric='ppl-big'
result_file=glob(f"{res_prefix}.{metric}")
result=pd.read_csv(result_file[0],header=None)
result=result.loc[common_edited_ixs]
metric_value=result[0].mean()
ppl_metrics.append(metric_value)

## fluency
metric='fluency'
result_file=glob(f"{res_prefix}.{metric}")
result=pd.read_csv(result_file[0],header=None)
result=result.loc[common_edited_ixs]
metric_value=result.loc[result[0]=='LABEL_1'].shape[0]/result.shape[0]
fluency_metrics.append(metric_value)


## toxicity
metric='toxicity'
result_file=glob(f"{res_prefix}.{metric}")
result=pd.read_json(result_file[0],lines=True)
result=unravel_toxicity_data(result)
result=result.loc[common_edited_ixs]
avg_toxicity=result['toxicity'].mean()
toxic_proba=result['toxicity'].apply(lambda x: 1 if x>0.5 else 0).mean()
avg_toxicity_metrics.append(avg_toxicity)
toxic_proba_metrics.append(toxic_proba)

## toxicity_int
metric='toxicity_int'
result_file=glob(f"{res_prefix}.{metric}")
result=pd.read_csv(result_file[0],header=None)
result=result.loc[common_edited_ixs]
avg_toxicity=result[0].mean()
toxic_proba=result[0].apply(lambda x: 1 if x>0.5 else 0).mean()
avg_toxicity_metrics_int.append(avg_toxicity)
toxic_proba_metrics_int.append(toxic_proba)

## repetitions
metric='repetitions'
result_file=glob(f"{res_prefix}.{metric}")
result=pd.read_json(result_file[0],lines=True)
result=result.loc[common_edited_ixs]
if result.empty:
    metric_value = 0
else:
    metric_value = result.loc[~result['repeated_phrase'].isna(),:].shape[0]/result.shape[0]
repetitions_metrics.append(metric_value)

## dist-3
bolt_subset=unravel(bolt)
bolt_subset=bolt_subset.loc[common_edited_ixs,].copy()
bolt_subset=ravel(bolt_subset)
_,_,dist3=distinctness(bolt_subset)
dist3_metrics.append(dist3)

Evaluating dist-n: 100%|██████████| 133/133 [00:00<00:00, 17707.04it/s]


In [18]:
print(ppl_metrics)
print(fluency_metrics)
print(avg_toxicity_metrics)
print(toxic_proba_metrics)
print(avg_toxicity_metrics_int)
print(toxic_proba_metrics_int)
print(repetitions_metrics)
print(dist3_metrics)

[10.678372690221408, 10.581806585147374, 10.564236049146672]
[0.9442508710801394, 0.9233449477351916, 0.9407665505226481]
[0.15134301969024377, 0.13708184573170742, 0.14154570252090598]
[0.03832752613240418, 0.03832752613240418, 0.041811846689895474]
[0.021489866550355462, 0.12628063179945198, 0.07015287145795393]
[0.017421602787456445, 0.03484320557491289, 0.03484320557491289]
[0, 0, 0.0]
[0.8664565580237976, 0.8674366263200263, 0.8688055444264908]


In [19]:
## putting all together
pd.DataFrame({'run_ids':['BOLT/detoxic/detoxic/replaced_vocab_roberta_for_jigsaw/',
                         'BOLT/detoxic/detoxic/locedit_em',
                         'BOLT/detoxic/detoxic/locedit_clsf'],
              'avg_toxicity_int':avg_toxicity_metrics_int,
              'toxic_proba_int':toxic_proba_metrics_int,    
              'avg_toxicity':avg_toxicity_metrics,
              'toxic_proba':toxic_proba_metrics,
              'ppl':ppl_metrics,
              'fluency_metrics':fluency_metrics,
              'dist-3':dist3_metrics,
              'rep_rate':repetitions_metrics})#.to_csv('new_module/toxicity_ablation_common_ixes.csv',index=False)

,run_ids,avg_toxicity_int,toxic_proba_int,avg_toxicity,toxic_proba,ppl,fluency_metrics,dist-3,rep_rate
0,BOLT/detoxic/detoxic/replaced_vocab_roberta_fo...,0.021490,0.017422,0.151343,0.038328,10.678373,0.944251,0.866457,0.0
1,BOLT/detoxic/detoxic/locedit_em,0.126281,0.034843,0.137082,0.038328,10.581807,0.923345,0.867437,0.0
2,BOLT/detoxic/detoxic/locedit_clsf,0.070153,0.034843,0.141546,0.041812,10.564236,0.940767,0.868806,0.0


In [20]:
## putting all together
pd.DataFrame({'run_ids':['BOLT/detoxic/detoxic/replaced_vocab_roberta_for_jigsaw/',
                         'BOLT/detoxic/detoxic/locedit_em',
                         'BOLT/detoxic/detoxic/locedit_clsf'],
              'avg_toxicity_int':avg_toxicity_metrics_int,
              'toxic_proba_int':toxic_proba_metrics_int,    
              'avg_toxicity':avg_toxicity_metrics,
              'toxic_proba':toxic_proba_metrics,
              'ppl':ppl_metrics,
              'fluency_metrics':fluency_metrics,
              'dist-3':dist3_metrics,
              'rep_rate':repetitions_metrics}).to_csv('new_module/toxicity_ablation_common_ixes_bolt.csv',index=False)

## Code to calculate metrics after dropping skipped sentences

In [73]:
import pandas as pd
from glob import glob
import os

os.getcwd()
os.chdir('/home/s3/hyeryung/mucoco')

from evaluation.prompted_sampling.evaluate import distinctness, repetition

In [74]:
run_ids="""duf2r40i
xo9cuxwt
rvz3rpkg
9wvrb6tm
rt1gftvo
c77e9d5d
gtgpwrna
7fjid6rd
vbwqb7sj
tsnyh5jg
bbja89dg
5de5l7xb
0v4eiwve
wtbdd1l0
59vupfa1
kpseb40i
r3neasla
gj7vpi4b
rc2h1xg6
0ejjmp0e
oc6pto51
wj9ww99a
gb09bmmt
nkss5p4q
by5s93vs
1p7censa""".split()

In [75]:
len(run_ids)

26

30개 세팅 
- 2개 model_tag X 15개 세팅
    - mucola
    - gbi x 2 (2 locate_methods)
    - combi x 2 (2 locate_methods)
    - mlm-rerankv0 x 2 (2 locate_methods)
    - mlm-rerankv1 x 2 (2 locate_methods) x 2 (2 selection criterion)
    - mlm-rerankv2 x 2 (2 locate_methods) x 2 (2 selection criterion)

In [76]:
## func to read output file
def unravel(outputs_df):
    outputs_df=outputs_df.explode('generations',ignore_index=True)
    
    outputs_df['prompt']=outputs_df['prompt'].apply(lambda x: x['text'])
    
    outputs_df['text']=outputs_df['generations'].apply(lambda x: x['text'])
    
    gen_dict=outputs_df['generations'].values[0]
    
    for col in gen_dict.keys():
        outputs_df[col] = outputs_df['generations'].apply(lambda x: x.get(col,None))

    return outputs_df

In [77]:
## get edited ixes
outputs_dfs=[]
edited_ixs=[]
for run_id in run_ids:

    output_file=[x for x in glob(f"outputs/toxicity/**/**/*{run_id}*/outputs_epsilon*.txt") if not x.endswith('filled.txt')]
    if len(output_file) == 0:
        output_file=[x for x in glob(f"outputs/toxicity/**/*{run_id}*/outputs_epsilon*.txt") if not x.endswith('filled.txt')]
    # print(output_file)
    outputs=pd.read_json(output_file[0], lines=True)

    if 'gbi' in output_file[0]:
        outputs=unravel(outputs)[['prompt','text','mucoco']].copy()
        outputs_dfs.append(outputs)
        edited_ixs.append(outputs.loc[outputs['mucoco'],].index.tolist())
    else:
        outputs=unravel(outputs)[['prompt','text','allsat']].copy()
        outputs_dfs.append(outputs)
        edited_ixs.append(outputs.loc[outputs['allsat']!=-1].index.tolist())

In [78]:
len(outputs_dfs), len(edited_ixs)

(26, 26)

In [95]:
# ## test hypothesis that explode in unravel preserves the order 

# eda=pd.merge(outputs_dfs[0],outputs_dfs[14],left_index=True,right_index=True)
# eda.loc[eda['allsat_x']!=-1, ['text_x','text_y']].values

In [8]:
metrics=['fluency','ppl-big','repetitions','sentiment','sentiment_int']

In [9]:
## ppl-big
metric='ppl-big'
ppl_metrics=[]
for i, run_id in enumerate(run_ids):

    result_file=glob(f"outputs/toxicity/**/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    if len(result_file) == 0:
        result_file=glob(f"outputs/toxicity/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    print(i, ': ', result_file[0])
    result=pd.read_csv(result_file[0],header=None)
    
    result=result.loc[edited_ixs[i]]
    metric_value=result[0].mean()
    ppl_metrics.append(metric_value)

0 :  outputs/toxicity/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds-energy-training/gbi-word-netps-1-nls-1-os200-es40-allsat-toxic-to-nontoxic-attention-duf2r40i/results_epsilon-1.09861228867-test.txt.ppl-big
1 :  outputs/toxicity/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/gbi-word-netps-1-nls-1-os200-es40-allsat-toxic-to-nontoxic-attention-xo9cuxwt/results_epsilon-1.09861228867-test.txt.ppl-big
2 :  outputs/toxicity/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds-energy-training/gbi-word-netps3-nls1-os20-es4-allsat-toxic-to-nontoxic-attention-rvz3rpkg/results_epsilon-1.09861228867-test.txt.ppl-big
3 :  outputs/toxicity/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/gbi-word-netps3-nls1-os20-es4-allsat-toxic-to-nontoxic-attention-9wvrb6tm/results_epsilon-1.09861228867-test.txt.ppl-big
4 :  outputs/toxicity/mlm-reranking/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds-energy-training/mlm-reranking-token-n

In [10]:
## fluency
metric='fluency'
fluency_metrics=[]
for i, run_id in enumerate(run_ids):

    result_file=glob(f"outputs/toxicity/**/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    if len(result_file) == 0:
        result_file=glob(f"outputs/toxicity/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    print(i, ': ', result_file[0])
    result=pd.read_csv(result_file[0],header=None)
    
    result=result.loc[edited_ixs[i]]
    metric_value=result.loc[result[0]=='LABEL_1'].shape[0]/result.shape[0]
    fluency_metrics.append(metric_value)

0 :  outputs/toxicity/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds-energy-training/gbi-word-netps-1-nls-1-os200-es40-allsat-toxic-to-nontoxic-attention-duf2r40i/results_epsilon-1.09861228867-test.txt.fluency
1 :  outputs/toxicity/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/gbi-word-netps-1-nls-1-os200-es40-allsat-toxic-to-nontoxic-attention-xo9cuxwt/results_epsilon-1.09861228867-test.txt.fluency
2 :  outputs/toxicity/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds-energy-training/gbi-word-netps3-nls1-os20-es4-allsat-toxic-to-nontoxic-attention-rvz3rpkg/results_epsilon-1.09861228867-test.txt.fluency
3 :  outputs/toxicity/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/gbi-word-netps3-nls1-os20-es4-allsat-toxic-to-nontoxic-attention-9wvrb6tm/results_epsilon-1.09861228867-test.txt.fluency
4 :  outputs/toxicity/mlm-reranking/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds-energy-training/mlm-reranking-token-n

In [11]:
## repetitions
metric='repetitions'
repetitions_metrics=[]
for i, run_id in enumerate(run_ids):

    result_file=glob(f"outputs/toxicity/**/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    if len(result_file) == 0:
        result_file=glob(f"outputs/toxicity/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    print(i, ': ', result_file[0])
    result=pd.read_json(result_file[0],lines=True)
    
    result=result.loc[edited_ixs[i]]
    if result.empty:
        metric_value = 0
    else:
        # print('!')
        metric_value = result.loc[~result['repeated_phrase'].isna(),:].shape[0]/result.shape[0]
    repetitions_metrics.append(metric_value)

0 :  outputs/toxicity/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds-energy-training/gbi-word-netps-1-nls-1-os200-es40-allsat-toxic-to-nontoxic-attention-duf2r40i/results_epsilon-1.09861228867-test.txt.repetitions
1 :  outputs/toxicity/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/gbi-word-netps-1-nls-1-os200-es40-allsat-toxic-to-nontoxic-attention-xo9cuxwt/results_epsilon-1.09861228867-test.txt.repetitions
2 :  outputs/toxicity/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds-energy-training/gbi-word-netps3-nls1-os20-es4-allsat-toxic-to-nontoxic-attention-rvz3rpkg/results_epsilon-1.09861228867-test.txt.repetitions
3 :  outputs/toxicity/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/gbi-word-netps3-nls1-os20-es4-allsat-toxic-to-nontoxic-attention-9wvrb6tm/results_epsilon-1.09861228867-test.txt.repetitions
4 :  outputs/toxicity/mlm-reranking/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds-energy-training/mlm-r

In [13]:
def unravel_toxicity_data(df):
    df['toxicity']=df['allresponses'].apply(lambda x: [x[0]['attributeScores']['TOXICITY']['summaryScore']['value'] for x in list(x.values())])
    df=df.explode('toxicity',ignore_index=True)
    return df

In [14]:
## repetitions
metric='toxicity'
avg_toxicity_metrics=[]
toxic_proba_metrics=[]
for i, run_id in enumerate(run_ids):

    result_file=glob(f"outputs/toxicity/**/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    if len(result_file) == 0:
        result_file=glob(f"outputs/toxicity/**/*{run_id}*/results_epsilon*-test.txt.{metric}")

    if metric in ['repetitions','toxicity']:
        result=pd.read_json(result_file[0],lines=True)
    else:
        result=pd.read_csv(result_file[0],header=None)

    if result.empty:
        result_file=glob(f"outputs/toxicity/**/**/*{run_id}*/results_filled.txt.{metric}")
        if len(result_file) == 0:
            result_file=glob(f"outputs/toxicity/**/*{run_id}*/results_filled.txt.{metric}")
        # print(result_file[0])
        if metric in ['repetitions','toxicity']:
            result=pd.read_json(result_file[0],lines=True)
        else:
            result=pd.read_csv(result_file[0],header=None)
    # print(result)
    result=unravel_toxicity_data(result)
    result=result.loc[edited_ixs[i]]
    avg_toxicity=result['toxicity'].mean()
    toxic_proba=result['toxicity'].apply(lambda x: 1 if x>0.5 else 0).mean()
    avg_toxicity_metrics.append(avg_toxicity)
    toxic_proba_metrics.append(toxic_proba)

In [15]:
## toxicity_int
metric='toxicity_int'
avg_toxicity_metrics_int=[]
toxic_proba_metrics_int=[]
for i, run_id in enumerate(run_ids):

    result_file=glob(f"outputs/toxicity/**/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    if len(result_file) == 0:
        result_file=glob(f"outputs/toxicity/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    # print(result_file[0])
    if metric in ['repetitions','toxicity']:
        result=pd.read_json(result_file[0],lines=True)
    else:
        result=pd.read_csv(result_file[0],header=None)
    
    result=result.loc[edited_ixs[i]]
    avg_toxicity=result[0].mean()
    toxic_proba=result[0].apply(lambda x: 1 if x>0.5 else 0).mean()
    avg_toxicity_metrics_int.append(avg_toxicity)
    toxic_proba_metrics_int.append(toxic_proba)

In [16]:
def ravel(unraveled_df):

    unraveled_df['generations']= unraveled_df.apply(lambda x: [{'text': x['text'],
                                                               'tokens': x['tokens']}],axis=1)
    return unraveled_df.groupby('prompt')['generations'].sum([]).reset_index()

In [17]:
## dist-3

dist3_metrics=[]
for i, run_id in enumerate(run_ids):
    
    result = outputs_dfs[i]
    result=result.loc[edited_ixs[i]]
    print(result.shape)
    result = ravel(result)
    print(result.shape)
    _,_,dist3=distinctness(result)
    dist3_metrics.append(dist3)

(520, 3)
(173, 2)


Evaluating dist-n: 100%|██████████| 173/173 [00:00<00:00, 7556.20it/s]


(328, 3)
(149, 2)


Evaluating dist-n: 100%|██████████| 149/149 [00:00<00:00, 10431.15it/s]


(520, 3)
(173, 2)


...

Evaluating dist-n: 100%|██████████| 149/149 [00:00<00:00, 13822.07it/s]


(520, 3)
(173, 2)


Evaluating dist-n: 100%|██████████| 173/173 [00:00<00:00, 12080.29it/s]


(328, 3)
(149, 2)


Evaluating dist-n: 100%|██████████| 149/149 [00:00<00:00, 14796.65it/s]


In [19]:
## putting all together
pd.DataFrame({'run_ids':run_ids,
              'avg_toxicity_int':avg_toxicity_metrics_int,
              'toxic_proba_int':toxic_proba_metrics_int,
              'avg_toxicity':avg_toxicity_metrics,
              'toxic_proba':toxic_proba_metrics,
              'ppl':ppl_metrics,
              'fluency_metrics':fluency_metrics,
              'dist-3':dist3_metrics,
              'rep_rate':repetitions_metrics}).to_csv('new_module/toxicity_ablation_dropped.csv',index=False)

In [18]:
pd.DataFrame({'run_ids':run_ids,
              'avg_toxicity_int':avg_toxicity_metrics_int,
              'toxic_proba_int':toxic_proba_metrics_int,
              'avg_toxicity':avg_toxicity_metrics,
              'toxic_proba':toxic_proba_metrics,
              'ppl':ppl_metrics,
              'fluency_metrics':fluency_metrics,
              'dist-3':dist3_metrics,
              'rep_rate':repetitions_metrics})

,run_ids,avg_toxicity_int,toxic_proba_int,avg_toxicity,toxic_proba,ppl,fluency_metrics,dist-3,rep_rate
0,duf2r40i,0.144238,0.021154,0.089080,0.003846,1.183877e+01,0.700000,0.745368,0.092308
1,xo9cuxwt,0.096598,0.045732,0.132274,0.039634,5.271674e+02,0.728659,0.693515,0.112805
2,rvz3rpkg,0.261650,0.111538,0.253697,0.101923,7.082493e+01,0.636538,0.868928,0.000000
3,9wvrb6tm,0.162628,0.125000,0.226851,0.082317,2.076643e+02,0.640244,0.805141,0.000000
4,rt1gftvo,0.122335,0.040385,0.090420,0.023077,1.896763e+02,0.778846,0.859186,0.000000
5,c77e9d5d,0.067335,0.039634,0.097130,0.021341,3.987478e+02,0.804878,0.798008,0.000000
6,gtgpwrna,0.122108,0.042308,0.092204,0.028846,1.871284e+02,0.771154,0.867484,0.000000
7,7fjid6rd,0.076454,0.051829,0.102324,0.027439,3.378623e+02,0.795732,0.797359,0.000000
8,vbwqb7sj,0.128610,0.051923,0.098750,0.030769,1.873398e+02,0.769231,0.866513,0.000000
9,tsnyh5jg,0.077896,0.051829,0.101706,0.027439,3.389895e+02,0.795732,0.795879,0.000000


## Count tokens decoded

In [125]:
set(edited_ixs[0]) == set(edited_ixs[4])

True

In [126]:
set(edited_ixs[1]) == set(edited_ixs[5])

True

In [106]:
origin=pd.read_json('new_module/data/toxicity-avoidance/testset_gpt2_2500.jsonl', lines=True)

In [108]:
origin=unravel(origin)

In [121]:
curr_ixs=edited_ixs[2]

In [122]:
curr_origin=origin.loc[curr_ixs,].copy()
curr_origin['num_toks']=curr_origin['tokens'].apply(len)
print(curr_origin['num_toks'].sum())

12340


In [127]:
curr_ixs=edited_ixs[5]

In [128]:
curr_origin=origin.loc[curr_ixs,].copy()
curr_origin['num_toks']=curr_origin['tokens'].apply(len)
print(curr_origin['num_toks'].sum())

7225


# Sentiment

## Code to calculate metrics on common examples

In [21]:
import pandas as pd
from glob import glob
import os
os.chdir('/home/s3/hyeryung/mucoco')
from evaluation.prompted_sampling.evaluate import distinctness, repetition
os.getcwd()

'/home/s3/hyeryung/mucoco'

In [22]:
run_ids="""ugr3ts53
7khk65lu
0ax3fe5q
745dg32u
9ujeyli7
hlnz0mx2
4tml0ox0
k7iftmov
z6iu47d2
ljuam185
a2bn8dk5
s13j6sqs
0xteb62f
qy6h3xm3
51eicj7v
0stmb3i8
m4u9glnx
bpfmw7l8
pygsotv9
4kzzg2vx
omhz1rr1
gahrb4gt
e30unz6h
3hra2btv
fxe95fbr
utoptt6f
ecoyl88a
fsputbpc
73xgem0e
3vt75up1""".split()

In [23]:
## func to read output file
def unravel(outputs_df):
    outputs_df=outputs_df.explode('generations',ignore_index=True)
    
    outputs_df['prompt']=outputs_df['prompt'].apply(lambda x: x['text'])
    
    outputs_df['text']=outputs_df['generations'].apply(lambda x: x['text'])
    
    gen_dict=outputs_df['generations'].values[0]
    
    for col in gen_dict.keys():
        outputs_df[col] = outputs_df['generations'].apply(lambda x: x.get(col,None))

    return outputs_df

In [24]:
def ravel(unraveled_df):

    unraveled_df['generations']= unraveled_df.apply(lambda x: [{'text': x['text'],
                                                               'tokens': x['tokens']}],axis=1)
    return unraveled_df.groupby('prompt')['generations'].sum([]).reset_index()

In [25]:
## get edited ixes
outputs_dfs=[]
edited_ixs=[]
common_edited_ixs=set()
for run_id in run_ids:

    output_file=[x for x in glob(f"outputs/sentiment/**/**/*{run_id}*/outputs_epsilon*.txt") if not x.endswith('filled.txt')]
    if len(output_file) == 0:
        output_file=[x for x in glob(f"outputs/sentiment/**/*{run_id}*/outputs_epsilon*.txt") if not x.endswith('filled.txt')]
    # print(output_file)
    outputs=pd.read_json(output_file[0], lines=True)

    if 'gbi' in output_file[0]:
        outputs=unravel(outputs)[['prompt','text','mucoco']].copy()
        outputs_dfs.append(outputs)
        edited_ixs.append(outputs.loc[outputs['mucoco'],].index.tolist())
        print(len(edited_ixs[-1]))
    else:
        outputs=unravel(outputs)[['prompt','text','allsat']].copy()
        outputs_dfs.append(outputs)
        edited_ixs.append(outputs.loc[outputs['allsat']!=-1].index.tolist())
        print(len(edited_ixs[-1]))
    
    if len(common_edited_ixs) == 0:
        common_edited_ixs = set(edited_ixs[-1])
    else:
        common_edited_ixs = common_edited_ixs.intersection(set(edited_ixs[-1]))

134
154
134
154
134
154
134
154
134
154
134
154
134
154
134
154
134
154
134
154
134
154
134
154
134
154
134
154
134
154


In [26]:
common_edited_ixs=list(common_edited_ixs)

In [27]:
len(common_edited_ixs)

113

In [130]:
# ## test hypothesis that explode in unravel preserves the order 

# eda=pd.merge(outputs_dfs[0],outputs_dfs[14],left_index=True,right_index=True)
# eda.loc[eda['allsat_x']!=-1, ['text_x','text_y']].values

In [131]:
metrics=['fluency','ppl-big','repetitions','sentiment','sentiment_int']

In [132]:
## ppl-big
metric='ppl-big'
ppl_metrics=[]
for run_id in run_ids:

    result_file=glob(f"outputs/sentiment/**/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    if len(result_file) == 0:
        result_file=glob(f"outputs/sentiment/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    # print(result_file[0])
    if metric in ['repetitions','toxicity']:
        result=pd.read_json(result_file[0],lines=True)
    else:
        result=pd.read_csv(result_file[0],header=None)
    
    result=result.loc[common_edited_ixs]
    metric_value=result[0].mean()
    ppl_metrics.append(metric_value)

In [133]:
## fluency
metric='fluency'
fluency_metrics=[]
for run_id in run_ids:

    result_file=glob(f"outputs/sentiment/**/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    if len(result_file) == 0:
        result_file=glob(f"outputs/sentiment/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    # print(result_file[0])
    if metric in ['repetitions','toxicity']:
        result=pd.read_json(result_file[0],lines=True)
    else:
        result=pd.read_csv(result_file[0],header=None)
    
    result=result.loc[common_edited_ixs]
    metric_value=result.loc[result[0]=='LABEL_1'].shape[0]/result.shape[0]
    fluency_metrics.append(metric_value)

In [134]:
## repetitions
metric='repetitions'
repetitions_metrics=[]
for run_id in run_ids:

    result_file=glob(f"outputs/sentiment/**/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    if len(result_file) == 0:
        result_file=glob(f"outputs/sentiment/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    # print(result_file[0])
    if metric in ['repetitions','toxicity']:
        result=pd.read_json(result_file[0],lines=True)
    else:
        result=pd.read_csv(result_file[0],header=None)
    
    result=result.loc[common_edited_ixs]
    if result.empty:
        metric_value = 0
    else:
        # print('!')
        metric_value = result.loc[~result['repeated_phrase'].isna(),:].shape[0]/result.shape[0]
    repetitions_metrics.append(metric_value)

In [135]:
## sentiment_ext
metric='sentiment_ext'
positive_proba_metrics_ext=[]
for run_id in run_ids:

    result_file=glob(f"outputs/sentiment/**/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    if len(result_file) == 0:
        result_file=glob(f"outputs/sentiment/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    # print(result_file[0])

    result=pd.read_json(result_file[0],lines=True)
    result=result.loc[common_edited_ixs]
    metric_value=result.loc[result['label']=='POSITIVE'].shape[0]/result.shape[0]
    positive_proba_metrics_ext.append(metric_value)
    

In [136]:
## sentiment_int
metric='sentiment_int'
positive_proba_metrics_int=[]
for run_id in run_ids:

    result_file=glob(f"outputs/sentiment/**/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    if len(result_file) == 0:
        result_file=glob(f"outputs/sentiment/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    # print(result_file[0])
    if metric in ['repetitions','sentiment_int']:
        result=pd.read_json(result_file[0],lines=True)
    else:
        result=pd.read_csv(result_file[0],header=None)
    result=result.loc[common_edited_ixs]
    metric_value=result.loc[result['label']=='LABEL_1'].shape[0]/result.shape[0]
    positive_proba_metrics_int.append(metric_value)
    

In [137]:
## dist-3

# distinctness()
dist3_metrics=[]
for run_id in run_ids:
    
    output_file=[x for x in glob(f"outputs/sentiment/**/**/*{run_id}*/outputs_epsilon*.txt") if not x.endswith('filled.txt')]
    if len(output_file) == 0:
        output_file=[x for x in glob(f"outputs/sentiment/**/*{run_id}*/outputs_epsilon*.txt") if not x.endswith('filled.txt')]
    # print(output_file)
    outputs=pd.read_json(output_file[0], lines=True)
    outputs=unravel(outputs)
    print(outputs.shape)
    outputs=outputs.loc[common_edited_ixs]
    print(outputs.shape)
    outputs=ravel(outputs)
    print(outputs.shape)
    _,_,dist3=distinctness(outputs)
    dist3_metrics.append(dist3)

(300, 9)
(113, 9)
(15, 2)


Evaluating dist-n: 100%|██████████| 15/15 [00:00<00:00, 6866.15it/s]


(300, 9)
(113, 9)
(15, 2)


Evaluating dist-n: 100%|██████████| 15/15 [00:00<00:00, 7058.74it/s]


(300, 9)
(113, 9)
(15, 2)


...

Evaluating dist-n: 100%|██████████| 15/15 [00:00<00:00, 6855.68it/s]


(300, 9)
(113, 9)
(15, 2)


Evaluating dist-n: 100%|██████████| 15/15 [00:00<00:00, 6922.06it/s]


(300, 9)
(113, 9)
(15, 2)


Evaluating dist-n: 100%|██████████| 15/15 [00:00<00:00, 7130.74it/s]


In [138]:
# ## putting all together


# pd.DataFrame({'run_ids':run_ids,
#               'positive_proba_int':positive_proba_metrics_int,
#               'positive_proba':positive_proba_metrics_ext,
#               'ppl':ppl_metrics,
#               'fluency_metrics':fluency_metrics,
#               'dist-3':dist3_metrics,
#               'rep_rate':repetitions_metrics}).to_csv('new_module/sentiment_ablation_common_ixes.csv',index=False)

In [139]:
# pd.DataFrame({'run_ids':run_ids,
#               'positive_proba_int':positive_proba_metrics_int,
#               'positive_proba':positive_proba_metrics_ext,
#               'ppl':ppl_metrics,
#               'fluency_metrics':fluency_metrics,
#               'dist-3':dist3_metrics,
#               'rep_rate':repetitions_metrics})

In [140]:
origin=pd.read_json('new_module/data/sentiment/outputs.txt.init.jsonl', lines=True)

In [141]:
## ppl-big
metric='ppl-big'
# ppl_metrics=[]
result_file=glob(f"new_module/data/sentiment/results-test-em.txt.{metric}")
if metric in ['repetitions','toxicity']:
    result=pd.read_json(result_file[0],lines=True)
else:
    result=pd.read_csv(result_file[0],header=None)

result=result.loc[common_edited_ixs]
metric_value=result[0].mean()
ppl_metrics.append(metric_value)

In [142]:
## fluency
metric='fluency'
# fluency_metrics=[]
result_file=glob(f"new_module/data/sentiment/results-test-em.txt.{metric}")
if metric in ['repetitions','toxicity']:
    result=pd.read_json(result_file[0],lines=True)
else:
    result=pd.read_csv(result_file[0],header=None)

result=result.loc[common_edited_ixs]
metric_value=result.loc[result[0]=='LABEL_1'].shape[0]/result.shape[0]
fluency_metrics.append(metric_value)

In [143]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained('/shared/s3/lab07/hyeryung/loc_edit/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds-2/step_2600_best_checkpoint/')

In [144]:
## repetitions
metric='repetitions'
# repetitions_metrics=[]
origin_subset=unravel(origin)
origin_subset=origin_subset.loc[common_edited_ixs,].copy()
origin_subset=ravel(origin_subset)
metric_value=repetition(origin_subset, tokenizer)
repetitions_metrics.append(metric_value)

Evaluating repetitions: 100%|██████████| 15/15 [00:00<00:00, 4099.47it/s]


In [145]:
## sentiment_ext
metric='sentiment_ext'
# positive_proba_metrics_ext=[]

result_file=glob(f"new_module/data/sentiment/results-test-em.txt.{metric}")

result=pd.read_json(result_file[0],lines=True)
result=result.loc[common_edited_ixs]
metric_value=result.loc[result['label']=='POSITIVE'].shape[0]/result.shape[0]
positive_proba_metrics_ext.append(metric_value)
    

In [146]:
## sentiment_int ## using EM
metric='sentiment_int'
# positive_proba_metrics_int=[]

result_file=glob(f"new_module/data/sentiment/results-test-em.txt.{metric}")

result=pd.read_json(result_file[0],lines=True)
result=result.loc[common_edited_ixs]
metric_value=result.loc[result['label']=='LABEL_1'].shape[0]/result.shape[0]
positive_proba_metrics_int.append(metric_value)
    

In [147]:
## dist-3
from evaluation.prompted_sampling.evaluate import distinctness
# distinctness()
# dist3_metrics=[]
origin_subset=unravel(origin)
origin_subset=origin_subset.loc[common_edited_ixs,].copy()
origin_subset=ravel(origin_subset)
_,_,dist3=distinctness(origin_subset)
dist3_metrics.append(dist3)

Evaluating dist-n: 100%|██████████| 15/15 [00:00<00:00, 4144.57it/s]


In [148]:
## putting all together


pd.DataFrame({'run_ids':run_ids+['original'],
              'positive_proba_metrics_int':positive_proba_metrics_int,
              'positive_proba_metrics_ext':positive_proba_metrics_ext,       
              'ppl':ppl_metrics,
              'fluency_metrics':fluency_metrics,
              'dist-3':dist3_metrics,
              'rep_rate':repetitions_metrics}).to_csv('new_module/sentiment_ablation_common_ixes.csv',index=False)

In [149]:
pd.DataFrame({'run_ids':run_ids+['original'],
              'positive_proba_metrics_int':positive_proba_metrics_int,
              'positive_proba_metrics_ext':positive_proba_metrics_ext,       
              'ppl':ppl_metrics,
              'fluency_metrics':fluency_metrics,
              'dist-3':dist3_metrics,
              'rep_rate':repetitions_metrics})

,run_ids,positive_proba_metrics_int,positive_proba_metrics_ext,ppl,fluency_metrics,dist-3,rep_rate
0,ugr3ts53,0.982301,0.849558,8.685479e+00,0.637168,0.620041,0.168142
1,7khk65lu,0.964602,0.902655,1.002011e+01,0.566372,0.641107,0.194690
2,0ax3fe5q,0.761062,0.460177,1.874492e+02,0.371681,0.899152,0.000000
3,745dg32u,0.796460,0.584071,2.940309e+02,0.212389,0.901199,0.000000
4,9ujeyli7,0.619469,0.433628,8.136595e+01,0.566372,0.895419,0.000000
5,hlnz0mx2,0.513274,0.460177,2.467734e+02,0.592920,0.895100,0.000000
6,4tml0ox0,0.539823,0.460177,7.612396e+01,0.504425,0.898264,0.000000
7,k7iftmov,0.398230,0.442478,1.275059e+02,0.495575,0.898297,0.000000
8,z6iu47d2,0.513274,0.442478,7.236399e+01,0.513274,0.897827,0.000000
9,ljuam185,0.407080,0.424779,9.704382e+01,0.469027,0.899421,0.000000


In [45]:
ppl_metrics=[]
fluency_metrics=[]
positive_proba_metrics_ext=[]
positive_proba_metrics_int=[]
repetitions_metrics=[]
dist3_metrics=[]

In [46]:
fpath='BOLT/sentiment/sentiment/replaced_vocab_roberta_for_yelp_polarity/pos.len20.jsonl'
res_prefix='BOLT/sentiment/sentiment/replaced_vocab_roberta_for_yelp_polarity/results-test.txt'

bolt=pd.read_json(fpath, lines=True)

## ppl-big
metric='ppl-big'
result_file=glob(f"{res_prefix}.{metric}")
result=pd.read_csv(result_file[0],header=None)
result=result.loc[common_edited_ixs]
metric_value=result[0].mean()
ppl_metrics.append(metric_value)

## fluency
metric='fluency'
result_file=glob(f"{res_prefix}.{metric}")
result=pd.read_csv(result_file[0],header=None)
result=result.loc[common_edited_ixs]
metric_value=result.loc[result[0]=='LABEL_1'].shape[0]/result.shape[0]
fluency_metrics.append(metric_value)

## sentiment_ext
metric='sentiment_ext'
result_file=glob(f"{res_prefix}.{metric}")
result=pd.read_json(result_file[0],lines=True)
result=result.loc[common_edited_ixs]
metric_value=result.loc[result['label']=='POSITIVE'].shape[0]/result.shape[0]
positive_proba_metrics_ext.append(metric_value)
    
## sentiment_int 
metric='sentiment_int'
result_file=glob(f"{res_prefix}.{metric}")
result=pd.read_json(result_file[0],lines=True)
result=result.loc[common_edited_ixs]
metric_value=result.loc[result['label']==2].shape[0]/result.shape[0] ## note the label. 2 : positive, 1: negative
positive_proba_metrics_int.append(metric_value)

## repetitions
metric='repetitions'
result_file=glob(f"{res_prefix}.{metric}")
result=pd.read_json(result_file[0],lines=True)
result=result.loc[common_edited_ixs]
if result.empty:
    metric_value = 0
else:
    metric_value = result.loc[~result['repeated_phrase'].isna(),:].shape[0]/result.shape[0]
repetitions_metrics.append(metric_value)

## dist-3
bolt_subset=unravel(bolt)
bolt_subset=bolt_subset.loc[common_edited_ixs,].copy()
bolt_subset=ravel(bolt_subset)
_,_,dist3=distinctness(bolt_subset)
dist3_metrics.append(dist3)

Evaluating dist-n: 100%|██████████| 15/15 [00:00<00:00, 8591.36it/s]


In [47]:
fpath='BOLT/sentiment/sentiment/locedit_em/pos.len20.jsonl'
res_prefix='BOLT/sentiment/sentiment/locedit_em/results-test.txt'

bolt=pd.read_json(fpath, lines=True)

## ppl-big
metric='ppl-big'
result_file=glob(f"{res_prefix}.{metric}")
result=pd.read_csv(result_file[0],header=None)
result=result.loc[common_edited_ixs]
metric_value=result[0].mean()
ppl_metrics.append(metric_value)

## fluency
metric='fluency'
result_file=glob(f"{res_prefix}.{metric}")
result=pd.read_csv(result_file[0],header=None)
result=result.loc[common_edited_ixs]
metric_value=result.loc[result[0]=='LABEL_1'].shape[0]/result.shape[0]
fluency_metrics.append(metric_value)

## sentiment_ext
metric='sentiment_ext'
result_file=glob(f"{res_prefix}.{metric}")
result=pd.read_json(result_file[0],lines=True)
result=result.loc[common_edited_ixs]
metric_value=result.loc[result['label']=='POSITIVE'].shape[0]/result.shape[0]
positive_proba_metrics_ext.append(metric_value)
    
## sentiment_int 
metric='sentiment_int'
result_file=glob(f"{res_prefix}.{metric}")
result=pd.read_json(result_file[0],lines=True)
result=result.loc[common_edited_ixs]
metric_value=result.loc[result['label']=='LABEL_1'].shape[0]/result.shape[0] ## note the label. 2 : positive, 1: negative
positive_proba_metrics_int.append(metric_value)

## repetitions
metric='repetitions'
result_file=glob(f"{res_prefix}.{metric}")
result=pd.read_json(result_file[0],lines=True)
result=result.loc[common_edited_ixs]
if result.empty:
    metric_value = 0
else:
    metric_value = result.loc[~result['repeated_phrase'].isna(),:].shape[0]/result.shape[0]
repetitions_metrics.append(metric_value)

## dist-3
bolt_subset=unravel(bolt)
bolt_subset=bolt_subset.loc[common_edited_ixs,].copy()
bolt_subset=ravel(bolt_subset)
_,_,dist3=distinctness(bolt_subset)
dist3_metrics.append(dist3)

Evaluating dist-n: 100%|██████████| 15/15 [00:00<00:00, 7448.15it/s]


In [48]:
fpath='BOLT/sentiment/sentiment/locedit_clsf/pos.len20.jsonl'
res_prefix='BOLT/sentiment/sentiment/locedit_clsf/results-test.txt'

bolt=pd.read_json(fpath, lines=True)

## ppl-big
metric='ppl-big'
result_file=glob(f"{res_prefix}.{metric}")
result=pd.read_csv(result_file[0],header=None)
result=result.loc[common_edited_ixs]
metric_value=result[0].mean()
ppl_metrics.append(metric_value)

## fluency
metric='fluency'
result_file=glob(f"{res_prefix}.{metric}")
result=pd.read_csv(result_file[0],header=None)
result=result.loc[common_edited_ixs]
metric_value=result.loc[result[0]=='LABEL_1'].shape[0]/result.shape[0]
fluency_metrics.append(metric_value)

## sentiment_ext
metric='sentiment_ext'
result_file=glob(f"{res_prefix}.{metric}")
result=pd.read_json(result_file[0],lines=True)
result=result.loc[common_edited_ixs]
metric_value=result.loc[result['label']=='POSITIVE'].shape[0]/result.shape[0]
positive_proba_metrics_ext.append(metric_value)
    
## sentiment_int 
metric='sentiment_int'
result_file=glob(f"{res_prefix}.{metric}")
result=pd.read_json(result_file[0],lines=True)
result=result.loc[common_edited_ixs]
metric_value=result.loc[result['label']=='LABEL_1'].shape[0]/result.shape[0] ## note the label. 2 : positive, 1: negative
positive_proba_metrics_int.append(metric_value)

## repetitions
metric='repetitions'
result_file=glob(f"{res_prefix}.{metric}")
result=pd.read_json(result_file[0],lines=True)
result=result.loc[common_edited_ixs]
if result.empty:
    metric_value = 0
else:
    metric_value = result.loc[~result['repeated_phrase'].isna(),:].shape[0]/result.shape[0]
repetitions_metrics.append(metric_value)

## dist-3
bolt_subset=unravel(bolt)
bolt_subset=bolt_subset.loc[common_edited_ixs,].copy()
bolt_subset=ravel(bolt_subset)
_,_,dist3=distinctness(bolt_subset)
dist3_metrics.append(dist3)

Evaluating dist-n: 100%|██████████| 15/15 [00:00<00:00, 8809.10it/s]


In [49]:
print(ppl_metrics)
print(fluency_metrics)
print(positive_proba_metrics_ext)
print(positive_proba_metrics_int)
print(repetitions_metrics)
print(dist3_metrics)

[8.0418128266987, 7.588677691627859, 7.362663391466612]
[0.9646017699115044, 0.9734513274336283, 0.9557522123893806]
[0.8672566371681416, 0.8849557522123894, 0.911504424778761]
[0.9823008849557522, 0.9911504424778761, 1.0]
[0, 0, 0]
[0.8343836736026324, 0.8475176702535454, 0.8255375600512661]


In [50]:
## putting all together
pd.DataFrame({'run_ids':['BOLT/sentiment/sentiment/replaced_vocab_roberta_for_yelp_polarity',
                         'BOLT/sentiment/sentiment/locedit_em',
                         'BOLT/sentiment/sentiment/locedit_clsf'],
              'positive_proba_metrics_ext': positive_proba_metrics_ext,
              'positive_proba_metrics_int': positive_proba_metrics_int,
              'ppl':ppl_metrics,
              'fluency_metrics':fluency_metrics,
              'dist-3':dist3_metrics,
              'rep_rate':repetitions_metrics})#.to_csv('new_module/toxicity_ablation_common_ixes.csv',index=False)

,run_ids,positive_proba_metrics_ext,positive_proba_metrics_int,ppl,fluency_metrics,dist-3,rep_rate
0,BOLT/sentiment/sentiment/replaced_vocab_robert...,0.867257,0.982301,8.041813,0.964602,0.834384,0
1,BOLT/sentiment/sentiment/locedit_em,0.884956,0.991150,7.588678,0.973451,0.847518,0
2,BOLT/sentiment/sentiment/locedit_clsf,0.911504,1.000000,7.362663,0.955752,0.825538,0


In [51]:
## putting all together
pd.DataFrame({'run_ids':['BOLT/sentiment/sentiment/replaced_vocab_roberta_for_yelp_polarity',
                         'BOLT/sentiment/sentiment/locedit_em',
                         'BOLT/sentiment/sentiment/locedit_clsf'],
              'positive_proba_metrics_ext': positive_proba_metrics_ext,
              'positive_proba_metrics_int': positive_proba_metrics_int,
              'ppl':ppl_metrics,
              'fluency_metrics':fluency_metrics,
              'dist-3':dist3_metrics,
              'rep_rate':repetitions_metrics}).to_csv('new_module/sentiment_ablation_common_ixes_bolt.csv',index=False)

In [69]:
## formality 
with open('data/formality/GYAFC_Corpus/Entertainment_Music/test/informal', 'r') as f:
    orig = [l.rstrip('\n') for l in f.readlines()]

In [72]:
tokenizer.encode(orig[0])

[3792, 4377, 14801, 16123, 317, 45328, 13]

In [80]:
orig_df=pd.DataFrame({'prompt': [{'text': '', 'tokens': []} for i in range(len(orig))],
              'generations': [[{'text': x, 'tokens': tokenizer.encode(x)}] for x in orig]}) 

In [77]:
os.makedirs('new_module/data/formality',exist_ok=True)

In [81]:
orig_df

,prompt,generations
0,"{'text': '', 'tokens': []}","[{'text': 'Is Any Baby Really A Freak.', 'toke..."
1,"{'text': '', 'tokens': []}",[{'text': 'aspen colorado has he best music fe...
2,"{'text': '', 'tokens': []}",[{'text': 'You can get almost anything on ebay...
3,"{'text': '', 'tokens': []}","[{'text': 'everybody is Dying to get in', 'tok..."
4,"{'text': '', 'tokens': []}",[{'text': 'not idiots like 50 cent and his who...
...,...,...
1411,"{'text': '', 'tokens': []}",[{'text': 'I LUBB IT.. [8] ILL KEEP U MY DIRRT...
1412,"{'text': '', 'tokens': []}",[{'text': 'I guess it is the blond from americ...
1413,"{'text': '', 'tokens': []}",[{'text': '(no offance) If you can answer that...
1414,"{'text': '', 'tokens': []}","[{'text': 'the song was called epic, i cant re..."


In [82]:
orig_df.to_json('new_module/data/formality/informal.jsonl',lines=True,orient='records')

## Code to calculate metrics after dropping skipped sentences

In [90]:
import pandas as pd
from glob import glob
import os

os.getcwd()
os.chdir('/home/s3/hyeryung/mucoco')

from evaluation.prompted_sampling.evaluate import distinctness, repetition

In [91]:
run_ids="""ugr3ts53
7khk65lu
0ax3fe5q
745dg32u
9ujeyli7
hlnz0mx2
4tml0ox0
k7iftmov
z6iu47d2
ljuam185
a2bn8dk5
s13j6sqs
0xteb62f
qy6h3xm3
51eicj7v
0stmb3i8
m4u9glnx
bpfmw7l8
pygsotv9
4kzzg2vx
omhz1rr1
gahrb4gt
e30unz6h
3hra2btv
fxe95fbr
utoptt6f
ecoyl88a
fsputbpc
73xgem0e
3vt75up1""".split()

In [92]:
len(run_ids)

30

30개 세팅 
- 2개 model_tag X 15개 세팅
    - mucola
    - gbi x 2 (2 locate_methods)
    - combi x 2 (2 locate_methods)
    - mlm-rerankv0 x 2 (2 locate_methods)
    - mlm-rerankv1 x 2 (2 locate_methods) x 2 (2 selection criterion)
    - mlm-rerankv2 x 2 (2 locate_methods) x 2 (2 selection criterion)

In [93]:
## func to read output file
def unravel(outputs_df):
    outputs_df=outputs_df.explode('generations',ignore_index=True)
    
    outputs_df['prompt']=outputs_df['prompt'].apply(lambda x: x['text'])
    
    outputs_df['text']=outputs_df['generations'].apply(lambda x: x['text'])
    
    gen_dict=outputs_df['generations'].values[0]
    
    for col in gen_dict.keys():
        outputs_df[col] = outputs_df['generations'].apply(lambda x: x.get(col,None))

    return outputs_df

In [94]:
## get edited ixes
outputs_dfs=[]
edited_ixs=[]
for run_id in run_ids:

    output_file=[x for x in glob(f"outputs/sentiment/**/**/*{run_id}*/outputs_epsilon*.txt") if not x.endswith('filled.txt')]
    if len(output_file) == 0:
        output_file=[x for x in glob(f"outputs/sentiment/**/*{run_id}*/outputs_epsilon*.txt") if not x.endswith('filled.txt')]
    # print(output_file)
    outputs=pd.read_json(output_file[0], lines=True)

    if 'gbi' in output_file[0]:
        outputs=unravel(outputs)[['prompt','text','mucoco']].copy()
        outputs_dfs.append(outputs)
        edited_ixs.append(outputs.loc[outputs['mucoco'],].index.tolist())
    else:
        outputs=unravel(outputs)[['prompt','text','allsat']].copy()
        outputs_dfs.append(outputs)
        edited_ixs.append(outputs.loc[outputs['allsat']!=-1].index.tolist())

In [95]:
# ## test hypothesis that explode in unravel preserves the order 

# eda=pd.merge(outputs_dfs[0],outputs_dfs[14],left_index=True,right_index=True)
# eda.loc[eda['allsat_x']!=-1, ['text_x','text_y']].values

In [96]:
metrics=['fluency','ppl-big','repetitions','sentiment','sentiment_int']

In [97]:
## ppl-big
metric='ppl-big'
ppl_metrics=[]
for i, run_id in enumerate(run_ids):

    result_file=glob(f"outputs/sentiment/**/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    if len(result_file) == 0:
        result_file=glob(f"outputs/sentiment/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    print(i, ': ', result_file[0])
    result=pd.read_csv(result_file[0],header=None)
    
    result=result.loc[edited_ixs[i]]
    metric_value=result[0].mean()
    ppl_metrics.append(metric_value)

0 :  outputs/sentiment/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds-energy-training/gbi-word-netps-1-nls-1-os200-es40-allsat-negative-to-positive-attention-ugr3ts53/results_epsilon-1.09861228867-test.txt.ppl-big
1 :  outputs/sentiment/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds/gbi-word-netps-1-nls-1-os200-es40-allsat-negative-to-positive-attention-7khk65lu/results_epsilon-1.09861228867-test.txt.ppl-big
2 :  outputs/sentiment/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds-energy-training/gbi-word-netps3-nls1-os20-es4-allsat-negative-to-positive-attention-0ax3fe5q/results_epsilon-1.09861228867-test.txt.ppl-big
3 :  outputs/sentiment/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds/gbi-word-netps3-nls1-os20-es4-allsat-negative-to-positive-attention-745dg32u/results_epsilon-1.09861228867-test.txt.ppl-big
4 :  outputs/sentiment/mlm-reranking/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds-energy-training/mlm-reran

In [98]:
## fluency
metric='fluency'
fluency_metrics=[]
for i, run_id in enumerate(run_ids):

    result_file=glob(f"outputs/sentiment/**/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    if len(result_file) == 0:
        result_file=glob(f"outputs/sentiment/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    print(i, ': ', result_file[0])
    result=pd.read_csv(result_file[0],header=None)
    
    result=result.loc[edited_ixs[i]]
    metric_value=result.loc[result[0]=='LABEL_1'].shape[0]/result.shape[0]
    fluency_metrics.append(metric_value)

0 :  outputs/sentiment/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds-energy-training/gbi-word-netps-1-nls-1-os200-es40-allsat-negative-to-positive-attention-ugr3ts53/results_epsilon-1.09861228867-test.txt.fluency
1 :  outputs/sentiment/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds/gbi-word-netps-1-nls-1-os200-es40-allsat-negative-to-positive-attention-7khk65lu/results_epsilon-1.09861228867-test.txt.fluency
2 :  outputs/sentiment/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds-energy-training/gbi-word-netps3-nls1-os20-es4-allsat-negative-to-positive-attention-0ax3fe5q/results_epsilon-1.09861228867-test.txt.fluency
3 :  outputs/sentiment/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds/gbi-word-netps3-nls1-os20-es4-allsat-negative-to-positive-attention-745dg32u/results_epsilon-1.09861228867-test.txt.fluency
4 :  outputs/sentiment/mlm-reranking/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds-energy-training/mlm-reran

In [99]:
## repetitions
metric='repetitions'
repetitions_metrics=[]
for i, run_id in enumerate(run_ids):

    result_file=glob(f"outputs/sentiment/**/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    if len(result_file) == 0:
        result_file=glob(f"outputs/sentiment/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    print(i, ': ', result_file[0])
    result=pd.read_json(result_file[0],lines=True)
    
    result=result.loc[edited_ixs[i]]
    if result.empty:
        metric_value = 0
    else:
        # print('!')
        metric_value = result.loc[~result['repeated_phrase'].isna(),:].shape[0]/result.shape[0]
    repetitions_metrics.append(metric_value)

0 :  outputs/sentiment/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds-energy-training/gbi-word-netps-1-nls-1-os200-es40-allsat-negative-to-positive-attention-ugr3ts53/results_epsilon-1.09861228867-test.txt.repetitions
1 :  outputs/sentiment/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds/gbi-word-netps-1-nls-1-os200-es40-allsat-negative-to-positive-attention-7khk65lu/results_epsilon-1.09861228867-test.txt.repetitions
2 :  outputs/sentiment/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds-energy-training/gbi-word-netps3-nls1-os20-es4-allsat-negative-to-positive-attention-0ax3fe5q/results_epsilon-1.09861228867-test.txt.repetitions
3 :  outputs/sentiment/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds/gbi-word-netps3-nls1-os20-es4-allsat-negative-to-positive-attention-745dg32u/results_epsilon-1.09861228867-test.txt.repetitions
4 :  outputs/sentiment/mlm-reranking/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds-energy-tr

In [100]:
## sentiment_ext
metric='sentiment_ext'
positive_proba_metrics_ext=[]
for i, run_id in enumerate(run_ids):

    result_file=glob(f"outputs/sentiment/**/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    if len(result_file) == 0:
        result_file=glob(f"outputs/sentiment/**/*{run_id}*/results_epsilon*-test.txt.{metric}")

    print(i, ': ', result_file[0])
    result=pd.read_json(result_file[0],lines=True)
    result=result.loc[edited_ixs[i]]
    print(result.shape)
    metric_value=result.loc[result['label']=='POSITIVE'].shape[0]/result.shape[0]
    positive_proba_metrics_ext.append(metric_value)
    

0 :  outputs/sentiment/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds-energy-training/gbi-word-netps-1-nls-1-os200-es40-allsat-negative-to-positive-attention-ugr3ts53/results_epsilon-1.09861228867-test.txt.sentiment_ext
(134, 2)
1 :  outputs/sentiment/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds/gbi-word-netps-1-nls-1-os200-es40-allsat-negative-to-positive-attention-7khk65lu/results_epsilon-1.09861228867-test.txt.sentiment_ext
(154, 2)
2 :  outputs/sentiment/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds-energy-training/gbi-word-netps3-nls1-os20-es4-allsat-negative-to-positive-attention-0ax3fe5q/results_epsilon-1.09861228867-test.txt.sentiment_ext
(134, 2)
3 :  outputs/sentiment/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds/gbi-word-netps3-nls1-os20-es4-allsat-negative-to-positive-attention-745dg32u/results_epsilon-1.09861228867-test.txt.sentiment_ext
(154, 2)
4 :  outputs/sentiment/mlm-reranking/roberta-base-yelp-sentiment

In [101]:
## sentiment_int
metric='sentiment_int'
positive_proba_metrics_int=[]
for i, run_id in enumerate(run_ids):

    result_file=glob(f"outputs/sentiment/**/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    if len(result_file) == 0:
        result_file=glob(f"outputs/sentiment/**/*{run_id}*/results_epsilon*-test.txt.{metric}")
    print(i, ': ', result_file[0])
    result=pd.read_json(result_file[0],lines=True)
    result=result.loc[edited_ixs[i]]
    metric_value=result.loc[result['label']=='LABEL_1'].shape[0]/result.shape[0]
    positive_proba_metrics_int.append(metric_value)
    

0 :  outputs/sentiment/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds-energy-training/gbi-word-netps-1-nls-1-os200-es40-allsat-negative-to-positive-attention-ugr3ts53/results_epsilon-1.09861228867-test.txt.sentiment_int
1 :  outputs/sentiment/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds/gbi-word-netps-1-nls-1-os200-es40-allsat-negative-to-positive-attention-7khk65lu/results_epsilon-1.09861228867-test.txt.sentiment_int
2 :  outputs/sentiment/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds-energy-training/gbi-word-netps3-nls1-os20-es4-allsat-negative-to-positive-attention-0ax3fe5q/results_epsilon-1.09861228867-test.txt.sentiment_int
3 :  outputs/sentiment/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds/gbi-word-netps3-nls1-os20-es4-allsat-negative-to-positive-attention-745dg32u/results_epsilon-1.09861228867-test.txt.sentiment_int
4 :  outputs/sentiment/mlm-reranking/roberta-base-yelp-sentiment-classifier-with-gpt2-large-embeds-e

In [102]:
def ravel(unraveled_df):

    unraveled_df['generations']= unraveled_df.apply(lambda x: [{'text': x['text'],
                                                               'tokens': x['tokens']}],axis=1)
    return unraveled_df.groupby('prompt')['generations'].sum([]).reset_index()

In [103]:
## dist-3

dist3_metrics=[]
for i, run_id in enumerate(run_ids):
    
    result = outputs_dfs[i]
    result=result.loc[edited_ixs[i]]
    print(result.shape)
    result = ravel(result)
    print(result.shape)
    _,_,dist3=distinctness(result)
    dist3_metrics.append(dist3)

(134, 3)
(15, 2)


Evaluating dist-n: 100%|██████████| 15/15 [00:00<00:00, 5880.41it/s]


(154, 3)
(15, 2)


Evaluating dist-n: 100%|██████████| 15/15 [00:00<00:00, 5217.23it/s]


(134, 3)
(15, 2)


...


Evaluating dist-n: 100%|██████████| 15/15 [00:00<00:00, 5206.87it/s]


(134, 3)
(15, 2)


Evaluating dist-n: 100%|██████████| 15/15 [00:00<00:00, 5729.40it/s]


(154, 3)
(15, 2)


Evaluating dist-n: 100%|██████████| 15/15 [00:00<00:00, 5607.86it/s]


In [16]:
## putting all together
pd.DataFrame({'run_ids':run_ids,
              'positive_proba_int':positive_proba_metrics_int,
              'positive_proba':positive_proba_metrics_ext,
              'ppl':ppl_metrics,
              'fluency_metrics':fluency_metrics,
              'dist-3':dist3_metrics,
              'rep_rate':repetitions_metrics}).to_csv('new_module/sentiment_ablation_dropped.csv',index=False)

In [104]:
pd.DataFrame({'run_ids':run_ids,
              'positive_proba_int':positive_proba_metrics_int,
              'positive_proba':positive_proba_metrics_ext,
              'ppl':ppl_metrics,
              'fluency_metrics':fluency_metrics,
              'dist-3':dist3_metrics,
              'rep_rate':repetitions_metrics})

,run_ids,positive_proba_int,positive_proba,ppl,fluency_metrics,dist-3,rep_rate
0,ugr3ts53,0.985075,0.873134,8.790101e+00,0.671642,0.624689,0.156716
1,7khk65lu,0.974026,0.902597,9.522580e+00,0.577922,0.618992,0.194805
2,0ax3fe5q,0.768657,0.522388,1.781145e+02,0.380597,0.897984,0.000000
3,745dg32u,0.824675,0.642857,2.565234e+02,0.207792,0.900569,0.000000
4,9ujeyli7,0.641791,0.477612,7.898115e+01,0.544776,0.894724,0.000000
5,hlnz0mx2,0.558442,0.519481,2.003867e+02,0.571429,0.895415,0.000000
6,4tml0ox0,0.589552,0.522388,7.317971e+01,0.507463,0.896832,0.000000
7,k7iftmov,0.493506,0.532468,1.108534e+02,0.500000,0.898100,0.000000
8,z6iu47d2,0.567164,0.492537,7.011548e+01,0.492537,0.896626,0.000000
9,ljuam185,0.474026,0.493506,8.816779e+01,0.474026,0.899170,0.000000
